<img src="images/cs5228-header-title.png" />

# Assignment 2 - Association Rule Mining & Tree-Base Models

Hello everyone, this assignment notebook covers Association Rule Mining (ARM) and tree-based models. There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e., your lines of code) between sentences "Your code starts here" and "Your code ends here". The space between these two lines does not reflect the required or expected lines of code. For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed).

When you work on this notebook, you can insert additional code cells (e.g., for testing) or markdown cells (e.g., to keep track of your thoughts). However, before the submission, please remove all those additional cells again. Thanks!

**Important:** 
* Remember to rename and save this Jupyter notebook as **A2_YourName_YourNUSNETID.ipynb** (e.g., **A2_BobSmith_e12345678.ipynb**) before submission!
* Remember to rename and save the Python script file **A2_YourName_YourNUSNETID.py** (e.g., **A2_BobSmith_e12345678.py**) before submission!
* Submission deadline is **Oct 23, 11.59 pm**. Late submissions will be penalized by 10% for each additional day. There is no need to use your full name if it's rather long; it's just important to easily identify you in Canvas etc.

Please also add your NUSNET and student id in the code cell below. This is just to make any identification of your notebook doubly sure.

In [ ]:
student_id = ''
nusnet_id = ''

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well documented, but you can use the overview below to double-check that you covered everything.

* **1 Association Rule Mining (ARM) (20 Points)**
    * 1.1 Implementing Apriori Algorithm (10 Points)
        * 1.1 a) Create Candidate Itemsets $L_k$ (6 Points)
        * 1.1 b) Generate Frequent Itemsets with Apriori Algorithm (4 Points)
    * 1.2 Recommending Movies using ARM (10 Points)
        * 1.2 a) Compare the Runs A-D and Discuss your Observations! (3 Points) 
        * 1.2 b) Compare the Runs A-D and Discuss the Results for Building a Recommendation Engine! (3 Points)
        * 1.2 c) Sketch a Movie Recommendation Algorithm Based on ARM (4 Points)
* **2 Tree Ensembles (30 Points)**
    * 2.1 Implementing a Random Forest Regressor (8 Points)
        * 2.1 a) Implementing Bagging (2 Points)
        * 2.1 b) Implementing Feature Sampling (2 Points)
        * 2.1 c) Training the Random Forest Regressor (2 Points)
        * 2.1 d) Predicting Output Values (2 Points)
    * 2.2 Implementing AdaBoost (12 Points)
        * 2.2 a) Training the AdaBoost Classifier (8 Points)
        * 2.2 b) Predicting Output Values (4 Points)
    * 2.3 Questions about Tree Ensembles (10 Points)
        * 2.3 a) Understanding Decision Tree Classifier (4 Points)
        * 2.3 b) Random Forest: Bagging Only vs. Bagging + Feature Sampling (2 Points)
        * 2.3 c) Random Forest: Regression vs. Classification (2 Points)
        * 2.3 d) AdaBoost for Classification (2 Points)

### Setting up the Notebook

In [ ]:
# Some magic so that the notebook will reload the external python script file any time you edit and save the .py file;
%load_ext autoreload
%autoreload 2

Making all the required imports:

In [ ]:
import sys
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from src.utils import *

**Important:** This notebook also requires you to complete in a separate `.py` script file. This keeps this notebook cleaner and simplifies testing your implementations for us. As you need to rename the file `A2.py`, you also need to edit the import statement below accordingly.

In [ ]:
from A2_SOLUTION import *
#from A2_BobSmith_e12345678 import * # <-- you well need to rename this accordingly

**Important:** Some tasks feature code cells to test your implementation and follow-up markdown cells with the expected outcomes. However, some implementations involve randomization. While the code cells set a fixed random seed, this only ensures the same result when running the code cell multiple times. However, it does *no* guarantee the result will be exactly the same as the expected outcome, since the results depend on the exact implementation which may differ between your and our solution. So do not worry if your results do not exactly match the expected outcome!

---

## 1 Association Rule Mining (ARM) (20 Points)

Your task is to implement the Apriori Algorithm for finding Association Rules. In more detail, we focus on the **Apriori Algorithm for finding Frequent Itemsets** -- once we have the Frequent Itemsets, we use a naive approach for the association rule. We will provide a small method for that part later.

### 1.1 Implementing Apriori Algorithm (10 Points)

#### Toy Dataset

The following dataset with 5 transactions and 6 different items is directly taken from the lecture slides. This should make it easier to test your implementation. The format is a list of tuples, where each tuple represents the set of items of an individual transaction. This format can also be used as input for the `efficient-apriori` package.

In [ ]:
transactions_demo = [
    ('bread', 'yogurt'),
    ('bread', 'milk', 'cereal', 'eggs'),
    ('yogurt', 'milk', 'cereal', 'cheese'),
    ('bread', 'yogurt', 'milk', 'cereal'),
    ('bread', 'yogurt', 'milk', 'cheese')
]

#### Auxiliary Methods

We want you to focus on the Apriori algorithm. So we provide a set of auxiliary functions. Feel free to look at their implementation in the file `src/utils.py`.

The method `unique_items()` returns all the unique items across all transactions.

In [ ]:
unique_items(transactions_demo)

The method `support()` calculates and returns the support for a given itemset and set of transactions.

In [ ]:
support(transactions_demo, ('bread', 'milk'))

The method `confidence()` calculates and returns the confidence for a given association rule and set of transactions. An association rule is represented by a 2-tuple, where the first element represents itemset X and the second element represents items Y (i.e., $X \Rightarrow Y$)

In [ ]:
confidence(transactions_demo, (('bread',), ('milk',)))

The method `merge_itemsets()` merges two given itemsets into one itemset.

In [ ]:
merge_itemsets(('bread', 'milk'), ('bread', 'eggs'))

For your implementation, you can make use of these auxiliary methods wherever you see fit. And that is, of course, strongly recommended, as it makes the programming task much easier. So, let's get started.

#### 1.1 a) Create Candidate Itemsets $L_k$ (6 Points)

Let's assume we have found $F_{k-1}$, i.e., all Frequent Itemsets for size $k-1$. For example $F_1$ is the set of all Frequent Itemsets of size 1, which is simply the set of unique items across all transactions with sufficient support. The next step is now to find $L_k$, all Candidate Itemsets of size $k$. In the lecture, we introduced two methods for this. For this assignment, we focus on the $\mathbf{F_{k-1} \times F_{k-1}}$ method -- that is, we use the Frequent Itemsets from the last step to calculate the Candidate Itemsets for the current step.

Recall from the lecture that creating $L_k$ involves two main parts:

* **Generating** all possible $k$-itemsets from the Frequent Itemsets $F_{k-1}$; and

* **Pruning** all $k$-itemsets that cannot be frequent based on the information we already have ($L_k$ should only contain the itemsets for which we indeed calculate the support for)


Recall that we also can (and should) **prune** any Candidate Itemsets than cannot possibly also be Frequent Itemsets  based on the information we already have. In other words, the Candidate Itemsets of size $k$ should only contain the itemsets for which we indeed calculate the support for.

**Hint:** In the lecture, to make it more illustrative, we first generate all possible Candidate Itemsets and then prune the ones that cannot possibly be frequent. In practice, to save memory space, it's better to check each Candidate Itemset immediately before even adding it to $L_k$. The skeleton code below reflects this. However, if you indeed want to implement pruning as its own step, you're free to do so.

**Implement method `generate_Lk()` to calculate the Candidate Itemsets $L_k$ given the Frequent Itemsets $F_{k-1}$!** Note that we walked in detail through an example of this process in the lecture. Below is a code cell that reflects this example to test your implementation.

In [ ]:
def generate_Lk(Fk_minus_one):

    # The code just looks a bit odd since we cannot get an element from a set using indexing
    k = len(next(iter(Fk_minus_one))) + 1

    # Initialize as set as a fail safe to avoid any duplicates
    Lk = set()
    
    for itemset1 in Fk_minus_one:
        for itemset2 in Fk_minus_one:
            
            ######################################################################
            ### Your code starts here ############################################
            
 

            ### Your code ends here ##############################################
            ######################################################################
            
            pass # Just there so the empty loop does not throw an error
    
    ######################################################################
    ### Your code starts here ############################################
    
    # MAY ONLY BE REQUIRED IF YOU TREAT PRUNING AS A SEPARATE STEP!!!
    
    ### Your code ends here ##############################################
    ###############################################################    
    return Lk

Test your code using the code cell below

In [ ]:
k_itemsets = generate_Lk({
    ('bread', 'cereal'), ('bread', 'milk'), ('bread', 'yogurt'), ('cereal', 'milk'),
    ('cereal', 'yogurt'), ('cheese', 'milk'), ('cheese', 'yogurt'), ('milk', 'yogurt')
})

for itemset in k_itemsets:
    print(itemset)

The output of the previous code cell should look as follows (although the order of the itemsets might be different):

```
('bread', 'milk', 'yogurt')
('bread', 'cereal', 'milk')
('cheese', 'milk', 'yogurt')
('cereal', 'milk', 'yogurt')
('bread', 'cereal', 'yogurt')
```

#### 1.1 b) Generate Frequent Itemsets with Apriori Algorithm (4 Points)

The method `generate_Lk()` covered the "Generate" and "Prune" steps of the Apriori Algorithm for finding Frequent Itemsets. Now only the "Calculate" and "Filter" step is missing. However, with `generate_Lk()` in place and together with the auxiliary methods we provide (see above), putting the Apriori Algorithm together should be pretty straightforward.

**Implement `frequent_itemsets_apriori()` to find all Frequent Itemset given a set of transactions and a minimum support of `min_support`!** Again, below is a code cell that reflects this example to test your implementation.

In [ ]:
def frequent_itemsets_apriori(transactions, min_support):
    
    # The frequent 1-itemsets are all unique items across all transactions with sufficient support
    # The one-liner below simply loops over all uniques items and checks the condition w.r.t. the support
    F1 = set([(s,) for s in unique_items(transactions) if support(transactions, (s,)) >= min_support ])
    
    # If there is not even a single 1-itemset that is frequent, we can just stop here
    if len(F1) == 0:
        return {}
    
    # Initialize dictionary with all current frequent itemsets for each size k
    # Example: { 1: {(a), (b), (c)}, 2: {(a, c), ...} }
    F = { 1: F1 }
    
    # Find now all frequent itemsets of size 2, 3, 4, ... (sys.maxsize basically mean infinity here)
    for k in range(2, sys.maxsize):

        Fk = set()
        
        ########################################################################################
        ### Your code starts here ##############################################################


                
        ### Your code ends here ################################################################
        ########################################################################################
                
        F[k] = Fk    

    # Merge the dictionary of itemsets to a single set and return it
    # Example: {1: {(a), (b), (c)}, 2: (a, c)} => {(a), (b), (c), (a, c)}
    return set.union(*[ itemsets for k, itemsets in F.items() ])

Test your code by running the following code cell.

In [ ]:
frequent_itemsets = frequent_itemsets_apriori(transactions_demo, 0.6)
for itemset in frequent_itemsets:
    print(itemset)

The output of the previous code cell should look as follows (although the order of the itemsets might be different):

```
('milk',)
('milk', 'yogurt')
('yogurt',)
('cereal', 'milk')
('bread',)
('bread', 'milk')
('bread', 'yogurt')
('cereal',)
```

#### From Frequent Itemsets to Association Rules (nothing for you to do here!)

Your implementation so far gives you the Frequent Itemsets in a list of transactions using the Apriori method. This step is typically the most time-consuming one in Association Rule Mining. However, we still have to do the second step and find all Association Rules given the Frequent Itemsets. We saw in the lecture that this can also be done in an efficient manner using the Apriori method to avoid checking all rules.

Since this step is typically less computationally expensive, we simply do it the naive way -- that is, we go over all Frequent Itemsets, and check for each Frequent Itemset and which of the Association Rules that can be generated from it has a sufficiently high confidence. With all the auxiliary methods we provide, this becomes trivial to implement, so we simply give you the method `find_association_rules()` below. Note how it uses your implementation of `frequent_itemsets_apriori()`.

In [ ]:
def find_association_rules(transactions, min_support, min_confidence):
    # Initialize empty list of association rules
    association_rules = []
    
    # Find and loop over all frequent itemsets
    for itemset in frequent_itemsets_apriori(transactions, min_support):
        if len(itemset) == 1:
            continue

        # Find and loop over all association rules that can be generated from the itemset
        for r in generate_association_rules(itemset):
            # Check if the association rule fulfils the confidence requriement
            if confidence(transactions, r) >= min_confidence:
                association_rules.append(r)
                
    # Return final list of association rules
    return association_rules

Let's execute the method over our small transaction dataset.

In [ ]:
for rule in find_association_rules(transactions_demo, 0.4, 1.0):
    print(rule)

#### Comparison with `efficient-apriori` package  (nothing for you to do here!)

You can run the apriori algorithm over the demo data to check if your implementation is correct. Try different values for the parameters `min_support` and `min_confidence` and compare the results. Note that the order of the returned association rules might differ between your implementation and the apriori one.

In [ ]:
_, rules = apriori(transactions_demo, min_support=0.4, min_confidence=1.0)

for r in rules:
    print('Rule [{} => {}] (support: {}, confidence: {}, lift: {})'.format(r.lhs, r.rhs, r.support, r.confidence, r.lift))

The `efficient-apriori` provides, of course, a much more efficient and convenient implementation (e.g., keeping track of all the metrics for each rule). And this is why we use this package for finding Association Rules in a real-world dataset below. Still, in its core, `efficient-apriori` implements the same underlying Apriori method to Find Frequent Itemsets (but also to find the Association Rules). If you're interested, further below, you can compare the runtimes of `efficient-apriori` and your implementation. Just don't be too disappointed :).

### 1.2 Recommending Movies using ARM (10 Points)

In this task, we look into using Association Rule Mining for recommending movies &mdash; more specifically, recommending movies on physical mediums (Blu-ray, DVD, etc.), assuming that is still a thing nowadays :).

**Dataset.** E-commerce sites do not really make their data publicly available, so we do not have any hard real-world dataset. For the context of this assignment, this is of course no problem. What we use here is a popular movie ratings dataset from [MovieLens](https://grouplens.org/datasets/movielens/). This dataset contains user ratings for movies (1-5 stars, incl. half stars, e.g., 3.5). More specifically, we use the [MovieLens 1M Dataset](https://grouplens.org/datasets/movielens/1m/) containing 1 Million ratings from ~6,000 users on ~4,000 movies and was released February 2003 -- so do not expect any recent Marvel movies :).

While using these ratings allow for more sophisticated recommendation algorithms &mdash; and we will look into some of those in a later lecture &mdash; here we are focusing on Association Rules. This includes that we need to convert this rating dataset into a transaction dataset, where a transaction represents all the movies a user has purchased. We already did this for you making the following assumption: A User has purchased all the movies s/he gave the highest rating. For example, if User A gave a highest rating of 4.5 to any movie, A has purchased all movies A rated with 4.5. This is certainly a simplifying assumption, but perfectly fine for this task here.

Let's have a quick look at the data. First, we load the ids and names of all movies into a dictionary. We need this dictionary since our transactions (i.e., the list of movies a user has bought) contains the ids and not the names of the movies. So to actually see the names of movies in the association rules, we need this way to map from a movie's id to its name.

In [ ]:
# Read file with movies (and der ids) into a pandas dataframe
df_movies = pd.read_csv('data/a2-arm-movies.csv', header=None)
# Convert dataframe to dictionary for quick lookups
movie_map = dict(zip(df_movies[0], df_movies[1]))
# Show the first 5 entries as example
for movie_id, movie_name in movie_map.items():
    print('{} -> {}'.format(movie_id, movie_name))
    if movie_id >= 5:
        break

Now we can load the transactions. Again, a transaction is a user's shopping history, i.e., all the movies the user has bought. 

In [ ]:
shopping_histories = []

# Read shopping histories; each line is a comma-separated list of the movies (i.e., their ids!) a user bought
with open('data/a2-arm-movie-shopping-histories.csv') as file:
    for line in file:
        shopping_histories.append(tuple([ int(i) for i in line.strip().split(',') ]))

# Show the shopping history of the first user for an example; we need movie_map to get the name of each movie
user = 0

print('Shopping history for user {} (used for Aprior algorithm)'.format(user))
print(shopping_histories[user])
print()
print('Detailed shopping history for user {}'.format(user))
for movie_id in shopping_histories[user]:
    print('{}: {}'.format(movie_id, movie_map[movie_id]))

With the dataset loaded, we are ready to find interesting Association Rules. For performance reasons, we use the `efficient_apriori` package &mdash; however, further below there is an optional code cell where you can use your own implementation of the Apriori algorithm, in case you are interested.

For added convenience, we provide method `show_top_rules()` which computes the Association Rules using the `efficient-apriori` package, but (a) sorts the rules w.r.t. the specified metric (default: lift), and (b) shows only the top-k rules (default: 5). The method also ensures a consistent output of each Association Rule. Each rule contains the LHS, RHS, as well as the support (s), confidence (c), and lift (l). Feel free to check out the code of method `show_top_rules()` in `src.utils` if anything might be unclear regarding its use.

**Run the following 4 code cells and interpret the results below!** All 4 code cells find Association Rules using the `efficient-apriori` package encapsulated in the auxiliary method `show_top_rules()` for convenience. Appreciate how Runs A-B differ with respect to the input parameter of the method calls! Also, note that we call `show_top_rules()` with `id_map=None` at first, so the results will only display the movie ids. Later, you will be asked to run the cells again with `id_map=movie_map` to see the actual names of the movies.

In [ ]:
%%time
# Run A
show_top_rules(shopping_histories, min_support=0.1, min_confidence=0.2, k=10, id_map=None)
#show_top_rules(shopping_histories, min_support=0.1, min_confidence=0.2, k=10, id_map=movie_map)

In [ ]:
%%time
# Run B
show_top_rules(shopping_histories, min_support=0.01, min_confidence=0.2, k=10, id_map=None)
#show_top_rules(shopping_histories, min_support=0.01, min_confidence=0.2, k=10, id_map=movie_map)

In [ ]:
%%time
# Run C
show_top_rules(shopping_histories, min_support=0.1, min_confidence=0.8, k=10, reverse=True, id_map=None)
#show_top_rules(shopping_histories, min_support=0.1, min_confidence=0.8, k=10, reverse=True, id_map=movie_map)

In [ ]:
%%time
# Run D
show_top_rules(shopping_histories, min_support=0.01, min_confidence=0.8, k=10, reverse=True, id_map=None)
#show_top_rules(shopping_histories, min_support=0.01, min_confidence=0.8, k=10, reverse=True, id_map=movie_map)

**Optional:** Feel free to uncomment and run the code cell below. It uses your implementation of the Apriori algorithm using the same parameters as Run C. You can use this code to double-check your implementation, but please be aware that it will run longer than the `efficient_apriori` package; although not too long for these parameters. Note that the result will not be in the same format and not sorted, but you can easily eyeball that the results will match the one of Run C above...or at least should :).

In [ ]:
#%%time

#rules = find_association_rules(shopping_histories, 0.1, 0.8)

#for lhs, rhs in rules:
#    print('{} => {}'.format(lhs, rhs))

#### 1.2 a) Compare the Runs A-D and Discuss your Observations! (3 Points)

You must have noticed numerous differences between the 4 runs A-D. List at least 3 differences you have found. You may want to consider the elapsed time and the resulting association rules. Briefly explain your observations! For this subtask, you do not need to look at the movie names (`id_map=None`) as your observations are not specific to the context of movie recommendations; at this we will look in 1.2 b)

**Your Answer:**

Now run the code cells above for Runs A-D again, but this time with `id_map=movie_map` so that the output will show for each rule the actual movie names.

#### 1.2 b) Compare the Runs A-D and discuss the results for building a recommendation engine! (3 Points)

Comparing the results of the different runs again, but now seeing the actual movie names, should give you some further insights how the choice of `min_support` and `min_confidence` might affect how the resulting rules are useful for building a recommendation engine.

**Your Answer:**

#### 1.2 c) Sketch a Movie Recommendation Algorithm Based on ARM (4 Points)

So far, we only looked at individual rules and how the set of rules changes for different parameter values for `min_support` and `min_confidence`. However, we still need some method like `make_recommendation(shopping_history)` that takes the shopping history of a user and returns 1 or more recommendations. The goal is here is *not* to implement such a method but outline the main concerns to consider when implementing such a method


(Hint: Do not forget that you not only have the information about Association Rules but also about the individual Frequent Itemsets)

**Your Answer:**

---

## 2 Tree Ensembles (30 Points)

With the implementation of a Decision Tree regressor in place, the goal of this task now is to show that the extension to ensemble models like Random Forests and AdaBoost is a rather straightforward one. In the following you will implement

* a Random Forest regressor *and*
* a AdaBoost classifier using Decision Trees

For both implementations you can directly adopt the algorithms covered in the lecture. Once you have completed this task, we hope that you see even more advanced models no longer as a scary black box :).
    
### 2.1. Implementing a Random Forest Regressor (8 Points)

We saw that a Random Forest trains a whole set of Decision Trees in parallel. To yield different Decision Trees each time, two sampling strategies are performed:

* **Bootstrap Sampling:** randomly sample N data points with replacement (N = total number of data points); *and*

* **Feature Sampling:** randomly choose only a subset of features to be used for training and prediction.


To test your implementation, let's first define a small toy dataset of 20 data samples. The two input features are the `weight` (in kg) and `height` (in cm) of a person, and the output value (i.e., the predicted variable) are the blood sugar level (in mmol/L).

In [ ]:
weights = np.array([68, 71, 92, 59, 80, 81, 75, 88, 45, 64, 59, 87, 80, 73, 55, 92, 93, 72, 49, 57])
heights = np.array([175, 175, 170, 168, 184, 184, 167, 155, 152, 163, 190, 161, 160, 174, 159, 183, 165, 181, 179, 154])

X_toy = np.stack((weights, heights), axis=1)
y_toy = np.array([7.8, 7.7, 11.0, 7.9, 6.8, 7.9, 6.3, 9.5, 8.1, 9.0, 6.0, 10.1, 7.0, 7.8, 7.7, 10.2, 9.8, 7.0, 6.4, 7.6])

print('Toy dataset -- #samples: {}, #features: {}'.format(X_toy.shape[0], X_toy.shape[1]))

Let's also load a small real-word dataset. We use the [Hitters](https://www.kaggle.com/floser/hitters) dataset which aims to predict the salaries of baseball players based on their statistics. You can check the website for more details about the different features. In the following, we just consider a subset of all features to keep it simple.

In [ ]:
subset = ['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Years', 'Assists', 'Errors']

df = pd.read_csv('data/a2-hitters.csv')
df = df.dropna(subset=subset+['Salary']) # We ignore all samples with NA values; it's not important here

X = df[subset].to_numpy()
y = df[['Salary']].to_numpy().squeeze()

# Note: sklearn.model_selection.train_test_split also shuffles the data!
# So we need to set random_state to ensure consistent results
X_hitters_train, X_hitters_test, y_hitters_train, y_hitters_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Size of training set: {}".format(len(X_hitters_train)))
print("Size of test: {}".format(len(X_hitters_test)))

#### 2.1 a) Implement Bagging (2 Points)

Implement method `bootstrap_sampling()` to generate a bootstrap sample for a given dataset! The input is represented by feature array `X`, and target array `y` containing the output values. You can use the code cell below to test your implementation. The cell computes five bootstrap samples and prints the shapes of data matrix `X_bootstrap` and output values vector `y_bootstrap`, as well as the vector of the first five samples output values.

In [ ]:
# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

my_random_forest = MyRandomForestRegressor()

for _ in range(5):
    X_bootstrap, y_bootstrap = my_random_forest.bootstrap_sampling(X_toy, y_toy)
    print(X_bootstrap.shape, y_bootstrap.shape, y_bootstrap[:10])

The expected output is as follows:

```
(20, 2) (20,) [ 7.  10.2  7.8  7.9  7.9  9.5  9.   7.6  6.4  6.8]
(20, 2) (20,) [ 9.   7.6  9.8  7.6  7.9 10.2 10.2  7.8  6.4  7.9]
(20, 2) (20,) [ 6.   7.9 10.1  6.4 11.   7.8  7.8  6.8  7.9  6.3]
(20, 2) (20,) [ 7.9  6.3 10.1  7.7  6.4  7.8  7.7  7.9  7.   6. ]
(20, 2) (20,) [10.2  7.9 10.2  7.8  9.8  7.   7.9  9.   7.9  7.8]
```

Most importantly, the size of each bootstrap sample should reflect the size of the original dataset -- that is, each bootstrap sample needs to contain 20 data samples, each with two features for the toy dataset (weight and height).

#### 2.1 b) Implement Feature Sampling (2 Points)

Implement method `feature_sampling()`! The input is a feature array `X`; to specify the number of features we use a simple approach where `max_features` specifies the ratio of features to be considered. Apart from the new dataset `X_sample` the method also returns the indices of the selected features; you will need this information when implementing the `fit()` method for training the Random Forest.

Have a look at the `__init__()` method; note that `MyRandomForestRegressor` already supports the `max_features` parameter we is aligned with the [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) from scikit-learn. For the implementation of `feature_sampling()`, however, we keep it simple, and consider `max_features` as a *float* values between 0 and 1, reflecting the ratio of features to included in the sample. For example, if `max_features=0.4`, only 40% of all features are part of the sample. If this number is not an integer, please always round up (Hint: [`np.ceil`](https://numpy.org/doc/stable/reference/generated/numpy.ceil.html)).

You can use the code cell below to test your implementation. Since the toy dataset has only 2 features, feature sampling with `max_features=0.5` and lower will return only one feature.

In [ ]:
# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

my_random_forest = MyRandomForestRegressor(max_features=0.5)

for _ in range(5):
    X_sample, indices_sampled = my_random_forest.feature_sampling(X_toy)
    print(X_sample.shape, indices_sampled)

The expected output is as follows:

```
(20, 1) [1]
(20, 1) [0]
(20, 1) [0]
(20, 1) [1]
(20, 1) [0]
```

Since we round up, the result will also be the same for, say `max_features=0.00001`. If you set `max_fetures` to a value larger than 0.5, all two features will be selected and each line in the output should be `(20, 2) [0 1]` or `(20, 2) [1 0]`. Values for `max_features` less or equal to `0`  or larger than `1.0` are of course invalid. But you do not need to make any checks in your implementation!

#### 2.1 c) Training the Random Forest (2 Points)

With **Bootstrap Sampling (Bagging)** and **Feature Sampling** -- and of course the implementation of your Decision Tree regressor -- everything is in place to finally train the Random Forest. As we saw in the lecture, training a Random Forest is simply training multiple Decision Trees based on different sampled datasets.

**Implement method `fit()` to train the Random Forest!** Have a good look at the given code snippet, and note that you need to keep track of the tuple `(regressor, indices_sampled)` for each estimator (i.e., Decision Tree). You will need `indices_sampled` later when making predictions, as you can only predict using those features that were also used during the training of a particular regressor.

You can use the code cell below to test your implementation.

In [ ]:
# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

my_random_forest = MyRandomForestRegressor(n_estimators=100, max_features=1.0).fit(X_toy, y_toy)

for i in range(5):
    print(my_random_forest.estimators[i][0].predict(np.array([[73, 180], [90, 170]])))

The expected output is as follows (might be somewhat different due to random effects not captured by seed):

```
[6.4 9.5]
[9.5 9.5]
[9.5 9.5]
[10.2 10.2]
[ 7. 11.]
```

#### 2.1 d) Predicting the Output Values (2 Points)

With a trained Random Forest, all that's left is to predict the output values for new data points. We do this by using each estimator (i.e., Decision Tree) to predict the value, and then calculate the average over predictions. Again, since we can only use those features with which an individual Decision Tree was trained on, you need to information about `indices_sampled` here (cf. `fit()` method).

**Implement method `predict()` to predict the output values for new data points! (2 Points)** The input is represented by a feature array `X` containing all new data points. If `X` contains N data points, the result should be an array containing all N predictions.

You can use the code cell below to test your implementation.

In [ ]:
# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

my_random_forest = MyRandomForestRegressor(n_estimators=100, max_features=1.0).fit(X_toy, y_toy)

print(my_random_forest.predict(np.array([[73, 180], [90, 170]])))

The expected output is as follows (might be slightly different due to random effects not captured by seed):

```
[ 7.023 10.486]
```

#### Additional Tests (nothing for you to do here!)

You have now implemented your Random Forest regressor. This means that you can now also compare your implementation with the one from scikit-learn. Due to the random sampling, it is basically not possible to ensure the same results. Although we `np.random.seed()` your implementation and the one from scikit-learn are just different for that to matter. To lower the effect of randomization let "switch off" feature sampling by always using all features (`max_features=1.0`).

Try different values for `n_estimators`. You should see that the more estimators you use the more similar the results. This shouldn't be surprising as we make the predictions as the means over a larger set of individual predictions from the estimators. Just don't go too high as your implementation is far from optimized :).

In [ ]:
%%time

# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

n_estimators = 100

my_random_forest = MyRandomForestRegressor(n_estimators=n_estimators, max_features=1.0).fit(X_hitters_train, y_hitters_train)
sk_random_forest = RandomForestRegressor(n_estimators=n_estimators, max_features=1.0).fit(X_hitters_train, y_hitters_train)

print(my_random_forest.predict(X_hitters_test)[:5])
print(sk_random_forest.predict(X_hitters_test)[:5])
print()

For `n_estimators = 100`, the expected output is as follows (might be slightly different due to random effects not captured by seed):

```
[292.89286 595.06663 316.865   520.7333  330.48   ]
[309.77143 580.30832 315.24833 547.77996 322.64   ]
```

### 2.2 Implementing AdaBoost (12 Points)

AdaBoost is a very popular ensemble technique that trains a series of *Weak Learners* to make predictions. Although AdaBoost is a generic technique, it is very commonly used with Decision Stumps as Weak Learners, since Decision Trees with a limited maximum height make naturally good Weak Learners. Again, we provide you with a skeleton code for the class implementing the AdaBoost Classifier. We call it `AdaBoostTreeClassifier` to avoid naming conflicts with `AdaBoostClassifier` of scikit-learn, and because we limit ourselves to Decision Trees (well, Stumps) as the estimators (i.e., the Weak Learners).

Since we build a classifier, we also need a small classification dataset. The classic [Iris dataset](https://archive.ics.uci.edu/dataset/53/iris) is one of the most well-known datasets in machine learning and statistics It contains 150 samples of iris flowers, evenly distributed among three species (*Iris setosa*, *Iris versicolor*, and *Iris virginica*). Each sample is described by 4 numerical features: sepal length, sepal width, petal length, and petal width, all measured in centimeters. Let's first load it into a file, convert the three string class labels into 0, 1, and 2, and shuffle the dataset

In [ ]:
np.random.seed(0)
df_iris = pd.read_csv('data/a2-iris.csv')
# Convert the 3 string class labels to 0, 1, and 2
df_iris.species = df_iris.species.factorize()[0]
df_iris = df_iris.sample(frac=1).reset_index(drop=True)
# Show sample of dataset
df_iris.head()

Now that all values are numeric, we can convert the DataFrame into NumPy arrays and generate the training and test set.

In [ ]:
X = df_iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()
y = df_iris['species'].to_numpy().squeeze()

training_size = int(0.8 * X.shape[0])

X_iris_train, y_iris_train = X[:training_size], y[:training_size]
X_iris_test, y_iris_test = X[training_size:], y[training_size:]

print("Size of training set: {}".format(len(X_iris_train)))
print("Size of test: {}".format(len(X_iris_test)))

#### 2.1 a) Training the AdaBoost Classifier (8 Points)

In the lecture, we went step by step through the training process of an AdaBoost classifier. We saw that this process comprises multiple but rather straightforward steps.


**Implement method `fit()`** to train your `AdaBoostTreeClassifier`. The skeleton code of method `fit()` allows you to focus on the core steps within each iteration for training the next Weak Learner (here, our Decision Stump). To help you a little bit, we list the main 4 steps and give you the first step -- training the next estimator using the current dataset sample -- for free.

**Important:** By default, `AdaBoostTreeClassifier` uses your implementation of `DecisionStumpClassifier` as its Weak Learner. In case you had problems implementing `DecisionStumpClassifier` or you simply want to test the results, you can also use scikit-learn's `DecisionTreeClassifier`. To make this change, just use the commented line under Step 1 to train the Weak Learner.

You can use the example calls below to test your implementation of the method.

In [ ]:
# We need to set the seed as the sampling is random
np.random.seed(0)

adaboost = AdaBoostTreeClassifier(n_estimators=5).fit(X_iris_train, y_iris_train)

print("The alpha values -- i.e., the amount-of-says -- are:")
print(adaboost.alphas)

The output of previous cell should be:

```
The alpha values -- i.e., the amount-of-says -- are:
[0.36544375 0.54110924 0.65028309 0.65364937 0.53673646]
```

#### 2.2 b) Predicting the Class Labels (4 Points)

As the last step, we now only need our AdaBoost classifier to predict the class labels for unseen data samples. Again, we saw in the lecture how this works: For each data sample, we check which of the `n_estimators` estimators predicts a certain class label, and the sum of all the alphas (i.e., the amounts of say) of the estimators of the same class.

The skeleton code of the `AdaBoostTreeClassifier` already provides a method `predict()` which takes a list of data samples as input and calls the method `predict_sample()` to predict the class label for each data sample individually. It's not that difficult to do this completely vectorized without the loop over the data samples, but here we want to focus on the basic algorithm and not worry about performance.

**Implement method predict_sample()** to predict the class label for a given data sample. Hint: Have again a look at [np.argwhere](https://numpy.org/doc/stable/reference/generated/numpy.argwhere.html) to maybe make your life easier. You can use the example calls below to test your implementation of the method.

In [ ]:
# We need to set the seed as the sampling is random
np.random.seed(0)

adaboost = AdaBoostTreeClassifier(n_estimators=5).fit(X_iris_train, y_iris_train)

print(adaboost.predict(X_iris_test))

The output of previous cell should be:

```
[0 2 0 0 2 0 2 1 1 1 2 2 1 2 0 1 2 2 0 1 1 2 1 0 0 0 2 1 2 0]
```

**Testing your Implementation on the IRIS Dataset.** We the code cell below, you can again directly compare the result your AdaBoost implementation with the scikit-learn [`AdaBoostClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html). Two things to note:

* By default, `AdaBoostClassifier` uses the `DecisionTreeClassifier` class as the estimators (i.e., the Weak Learners) with `max_depth=1`

* By default, `AdaBoostClassifier` sets `n_estimators=50`; we therefore choose the same default value for `AdaBoostTreeClassifier`. Note that for this default value, both your and the sklearn implementation should return the same result. However, the result might be slightly different for different values for `n_estimator`.

As a result, we do not have to set any parameters for `AdaBoostClassifier` and can use the default ones to allow for a fair comparison with your implementation of `AdaBoostTreeClassifier`.

In [ ]:
my_adaboost = AdaBoostTreeClassifier().fit(X_iris_train, y_iris_train)
sk_adaboost = AdaBoostClassifier().fit(X_iris_train, y_iris_train)

my_y_pred = my_adaboost.predict(X_iris_test)
sk_y_pred = sk_adaboost.predict(X_iris_test)

my_f1 = f1_score(y_iris_test, my_y_pred, average='macro')
sk_f1 = f1_score(y_iris_test, sk_y_pred, average='macro')

print('The f1 score of your AdaBoost implementation on the IRIS dataset is {:.3f}'.format(my_f1))
print('The f1 score of the sklearn AdaBoost implementation on the IRIS dataset is {:.3f}'.format(sk_f1))

print(my_y_pred)
print(sk_y_pred)
print(sk_y_pred-my_y_pred)

You should see an f1 score of **0.933** using both your implementation as well as the one from scikit-learn.

### 2.3 Questions about Tree Ensembles (10 Points)

#### 2.3 a) Understanding Decision Tree Classifier (4 Points)

The plot below shows a toy dataset of $6\times 8 = 64$  data points for a binary classification task. Each data point belongs either to Class Red (big dots) or Blue (small dots); the different dot sizes are only there to accommodate color-blind students, if needed.

<img src="images/a2-example-data-distribution-02.png">

Now imagine you train a full Decision Tree (e.g., by running `sklearn.tree.DecisionTreeClassifier`) over this dataset without any early stopping, pruning, etc. Note that this implementation of a Decision Tree performs only binary splits.

**Complete the following table by answering each question and providing a brief explanation? (3 Points)** In some of the questions, you are asked to flip the class labels of 2 of the data points. To specify a data point you can use `(x,y)` coordinates. For example, for flipping the class labels of the bottom-left data point and top-left data point, please write `(1,1)<=>(1,6)`.
 
**Your Answer:**

This is a markdown cell. Please fill in your answers for (1)~(6).

| No. | Question                                                                                                   | Answer       | Brief Explanation |
|-----|------------------------------------------------------------------------------------------------------------|--------------| ------- |
| (1)  | What is the number of splits of the Decision Tree trained over the orginal dataset given about?                                                                              | ??? | ??? |
| (2)  | You can now flip the class labels of 2 data points. Which 2 pairs of data points do you pick and flip the labels to minimize the number of nodes in the new resulting Decision Tree? What will be the number of splits in the new Decision Tree (compared to the original dataset)? |??? | ???</font> |
| (3)  | You can now change the class labels of 1 Red data point in the original data distribution. Which Red data point do you pick and change its label to maximize the number of nodes in the new resulting Decision Tree? What will be the number of splits in the new Decision Tree  (compared to the original dataset)? | ??? | ???</font> |
| (3)  | You can now arbitrarily change the class labels of *all* data points. How would you label the data points if your goal would be to maximize the number of required splits when training a full Decision Tree classifier? | ??? | ??? |

#### 2.3 b) Random Forest: Bagging Only vs. Bagging + Feature Sampling (2 Points)

The code cell below trains a two series of 20 Decision Trees each. One series uses only Bagging (i.e., Bootstrap Sampling) for the training data; the other series uses both Bagging and Feature Sampling. The output shows the results for "Bagging only" in the left column and "Bagging + Feature Sampling" in the right column. `root index` represents the feature index chosen as the root node (i.e., for the first split); `#nodes` represents the total number of nodes in the trained Decision Tree.

**Note:** There is nothing for you to implement here, but feel free to increase the number of Decision Trees beyond 20.

In [ ]:
# We need to set the seed as the sampling is random, and we want to ensure consistent results
np.random.seed(0)

my_random_forest_bagging = MyRandomForestRegressor(max_features=1.0)
my_random_forest_sampling = MyRandomForestRegressor(max_features=0.2)

print("Bagging only\t\t\t\tBagging + Feature Sampling")
for _ in range(20):
    # Create a new bootstrap sample
    X_t, y_t = my_random_forest_bagging.bootstrap_sampling(X_hitters_train, y_hitters_train)
    regressor_bagging = DecisionTreeRegressor().fit(X_t, y_t)
    
    X_t, indices_sampled = my_random_forest_sampling.feature_sampling(X_t)
    regressor_sampling = DecisionTreeRegressor().fit(X_t, y_t)    
    
    # Print core features of trained Decision Tree
    # (feature index of root node, total of number of nodes in Decision Tree)
    print('#root index: {},  #nodes: {}\t\t#root index: {},  #nodes: {}'
          .format(regressor_bagging.tree_.feature[0], regressor_bagging.tree_.node_count,
                  indices_sampled[regressor_sampling.tree_.feature[0]], regressor_sampling.tree_.node_count))

**Interpret the result!** Comparing the resulting series of Decision Trees when using **Bagging only** and **Bagging + Feature Sampling**, what differences and noteworthy details can you observe and what insights into the dataset can you gain from your observations. List all your observations together with a brief explanation!

(It might be useful to remember that the size of `X_train` is 210 data samples with 8 features; note also that each [`sklearn.tree.DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) has been trained with their default parameters)

**Your Answer:**

#### 2.3 c) Random Forest: Regression vs. Classification (2 Points)

The markdown cell below shows a screenshot showing similar results as you have seen in 2.3 a). The only difference here is that these results stem from a classification task (and not a regression task). More specifically, the simple [IRIS](https://archive.ics.uci.edu/ml/datasets/iris) dataset was: it's small and clean, and has only numerical features. However, since we did not implement a Random Forest Classifier, we just look a screenshot of results.

<img src="images/a2-rf-regression-vs-classification.png">

**Interpret the result!** Again, what differences and noteworthy details can you observe and what insights into the dataset can you gain from your observations. Particularly, compare these results with the results from the regression task in 2.3 a). List all your observations together with a brief explanation!

**Your Answer:**

#### 2.3 d) AdaBoost for Classification (2 Points)

Assume you use your implementation of `AdaBoostTreeClassifier` to train a binary classifier of the dataset shown in 3.2 a). Will the binary classifier be able to achieve a training error (not test error!) of 0? Explain your answer! (Your explanation is more important than a simple Yes/No answer)

**Your Answer:**